In [1]:
import docx2txt
import xlrd
import pandas as pd
import re 
import numpy as np
import os
from tqdm import tqdm
import sys
import copy

In [2]:
# option_sheet_name = 'Sheet1'
# option_sheet_index = 0

option_sheet_name = 'Sheet2'
option_sheet_index = 1


In [3]:
tmp_file_path = r"./tmp.txt"

In [4]:
excel = pd.read_excel(r"./data/decoration/装修手册_故障代码.xls",header=None,sheet_name=option_sheet_name)
excel

,0,1,2,3,4,5,6
0,H-1 整个工作装置、回转、和行走的速度或动力低,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,故障,· 整个工作装置、回转、和行走的速度或动力低。,NaN,NaN,NaN,NaN,NaN
3,相关信息,· 在工作模式为快速模式（P）时，进行所有故障诊断。,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,起因,NaN,正常状态下的标准值/故障诊断备注,NaN,NaN,NaN
6,NaN,NaN,NaN,★ 关闭发动机，做好准备，然后在发动机全速时进行故障诊断。,NaN,NaN,NaN
7,NaN,1,卸荷阀故障,操纵杆,NaN,卸载压力,NaN
8,NaN,NaN,NaN,所有操纵杆处于中位,NaN,3.5 ± 1.0 Mpa,NaN
9,NaN,NaN,NaN,NaN,NaN,{36 ± 10 kg/cm2},NaN


In [7]:
workbook = xlrd.open_workbook(r"./data/decoration/装修手册_故障代码.xls",formatting_info=True)

sheet = workbook.sheet_by_index(option_sheet_index)

In [36]:
synonymMap = {'报警级别':['报警级别'], '故障代码':['故障代码'], '故障':['故障'], '故障内容':['故障内容'],  \
              '控制器的反应':['控制器的反应'], '机器发生的故障':['机器发生的故障'],  \
             '相关信息':['相关信息'], '原因':['原因'], '诊断操作':['诊断操作'], '可能的原因和 标准':['可能的原因和 标准']}

name_to_en = {"故障代码":"faultCode", "故障名称":"faultName", "报警级别":"alertLevel","故障内容":"faultContent", 
                 "控制器的反应":"controllerReaction", "机器发生的故障":"machineFault", "相关信息":"relatedInfo", 
                 "起因":"cause",
                  "诊断操作":"diagnosisAction", "标准值":"standardVal", "附图":"image", 
                  "故障部位系统":"monitorSystem", "故障":"faultName", "步骤数":"stepNum"}
name_to_en = {k: v.lower() for k, v in name_to_en.items()}
name_to_ch = {v: k for k, v in name_to_en.items()}
name_to_ch

{'faultcode': '故障代码',
 'faultname': '故障',
 'alertlevel': '报警级别',
 'faultcontent': '故障内容',
 'controllerreaction': '控制器的反应',
 'machinefault': '机器发生的故障',
 'relatedinfo': '相关信息',
 'cause': '起因',
 'diagnosisaction': '诊断操作',
 'standardval': '标准值',
 'image': '附图',
 'monitorsystem': '故障部位系统',
 'stepnum': '步骤数'}

## 切分出每个故障

s = 'H-2 发动机转速极速下降或发动机突然熄火'
re.match(r'([E|H]-\d+)\s*(.*)',s).group(2)

In [9]:
# 适用于E和H的故障代码
# 手工处理了H-10，H-23，H-24，H-27，E-2（把3个子故障分成了3个故障）E-1，E-3（原因及处理方案栏分成了多个表格）
code_line_num = []
for i in range(len(excel)):
#     print(i)
#     re.match(r'故障代码 \[\w*\].*',str(excel[i][0])) is not None:
    match_result = re.match(r'([E|H]-[\d|_]+)\s*(.*)',str(excel.loc[i,0])) 
    if match_result is not None:
#         print(excel.loc[i,0])
#         print(match_result.group(1), match_result.group(2))
        code_line_num.append(i)
code_line_num.append(len(excel))
print(len(code_line_num))
code_line_num
        

70


[0,
 49,
 85,
 108,
 121,
 145,
 174,
 203,
 224,
 241,
 253,
 269,
 284,
 302,
 312,
 327,
 353,
 363,
 373,
 412,
 447,
 481,
 512,
 531,
 576,
 596,
 616,
 631,
 652,
 669,
 683,
 717,
 737,
 750,
 1079,
 1337,
 1407,
 1426,
 1493,
 1529,
 1590,
 1625,
 1694,
 1740,
 1754,
 1765,
 1793,
 1827,
 1846,
 1880,
 1890,
 1901,
 1911,
 1976,
 2083,
 2155,
 2192,
 2273,
 2335,
 2344,
 2353,
 2363,
 2372,
 2381,
 2391,
 2401,
 2412,
 2485,
 2512,
 2542]

In [41]:
len(code_line_num)

70

In [40]:
for line_num in code_line_num:
    print(excel.loc[line_num,0])

H-1 整个工作装置、回转、和行走的速度或动力低
H-2 发动机转速极速下降或发动机突然熄火
H-3 工作装置、回转和行走系统不工作
H-4 液压泵周围出现异常声音
H-5 精细控制性能或响应低
H-6 动臂速度或动力低
H-7 斗杆速度或动力低
H-8 铲斗速度或动力低
H-9 工作装置在单项操作中不动
H-10_1 工作装置的自然下降量大
H-10_2 工作装置的自然下降量大
H-10_3 工作装置的自然下降量大
H-11 工作装置的时滞长
H-12 一个工作装置溢流时，其它工作装置移动
H-13 点触式加力功能不工作
H-14 在工作装置的复合操作中，载荷较重的装置移动慢
H-15 在回转和动臂提升的复合操作中，动臂提升速度慢
H-16 在回转和行走的复合操作中，行走速度急剧下降
H-17 机器不能直线行走
H-18 行走速度慢
H-19 机器转向困难或行走动力低
H-20 行走速度不变或行走速度低/高
H-21 一侧履带不运转
H-22 上部回转平台不能回转
H-23_1 回转加速性能差或回转速度慢
H-23_2 回转加速性能差或回转速度慢
H-24_1 上部回转平台停止回转时严重超速
H-24_1 上部回转平台停止回转时严重超速
H-25 上部回转平台停止回转时震动大
H-26 上部回转平台停止回转时噪音大
H-27 回转的漂移量大
H-28 附加装置油路无法改变
H-29 附加装置油路流量无法控制
E-1 发动机不起动（发动机不转）
E-2 预热器不工作
E-3 起动开关转到 ON 位置时，机器监控器不显示信息
E-4 起动开关转到 ON 位置时（发动机停机），基本检查监控器点亮
E-5 发动机运行时，预警监控器点亮
E-6 发动机运行时，紧急停止监控器点亮
E-7 燃油油位表的显示不正确
E-8 发动机冷却液温度计的显示不正确
E-9 液压油温度计的显示不正确
E-10 机器监控器上的显示和实际机器不同
E-11 机器监控器屏的某些区域不显示
E-12 功能开关不工作
E-13 自动预热系统不工作（寒冷季节时）
E-14 自动降速工作不正常
E-15 工作模式不改变
E-16 行走速度不改变
E-17 报警蜂鸣器无法取消
E-18 起动开关转到 ON 位置时，不显示小时计
E-19 无法选择服务模式
E-20 工作装置、回转和行走不工作或无法锁定
E-

KeyError: 'the label [2542] is not in the [index]'

## 逐个读取出每个实体/属性的值

In [31]:
# 适用于一般的故障代码
fc_dicts = {}
for i_code in range(len(code_line_num)-1):
#     if i_code == 3:
#         break
    fc_dict = {}
    i_start = code_line_num[i_code]
    i_end = code_line_num[i_code+1]
#     print(str(excel.loc[i,0]))
    match_result = re.match(r'([E|H]-[\d|_]+)\s*(.*)',str(excel.loc[i_start,0])) 
    fc_code = match_result.group(1)
    fc_content = match_result.group(2)
    fc_dict['故障代码'] = fc_code
    fc_dict['故障'] = fc_content
    print("i range in (%d,%d)"%(i_start, i_end))
#     print(excel.loc[i_start:i_end])

    for i in range(i_start, i_end):
        first_col_content = excel.loc[i,0]
        if first_col_content in synonymMap.keys():
#             print(first_col_content)
            if first_col_content == '报警级别':
                fc_dict[first_col_content] = excel.loc[i+1,0]
                for j in range(1,excel.shape[1]):
                    if excel.loc[i,j] == '故障代码':
#                         print(i+1)
#                         print(excel.loc[i+1,j])
                        fc_code_2 = re.match(r'\[?([\w|@]*)\]?',str(excel.loc[i+1,j])).group(1)
                        assert fc_code == fc_code_2, "%s %s"%(fc_code, fc_code_2)
                        fc_dict[excel.loc[i,j]] = fc_code
                    if excel.loc[i,j] == '故障':
                        if fc_content == excel.loc[i,j+1]:
                            fc_dict[excel.loc[i,j]] = fc_content
                        else:
                            print("WARNING：故障内容前后不一致：line_num=%d %s %s"%(i,fc_content, excel.loc[i,j+1]))
                            fc_dict[excel.loc[i,j]] = fc_content
#                         assert , 
                        
                        fc_dict['故障部位系统'] = excel.loc[i+1,j+1]
            if first_col_content == '故障内容':
#                 print(excel.shape)
#                 print(i_content)
#                 print(i)
#                 print(excel.loc[i_content,1])
                i_content = i 
                if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                    content = excel.loc[i,1]
                else:
                    content = excel.loc[i,2]
                fc_dict[first_col_content] = content
            if first_col_content == '控制器的反应':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now,0] != '机器发生的故障':
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '机器发生的故障':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now+1,0] != '相关信息': # ‘相关信息’那几个字总是偏下一栏
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '相关信息':
                
                for i_content_start in range(i-1,i-3,-1):
                    cell = sheet.cell(i_content_start,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
#                     print(dump_dic)
                    if dump_dic['bottom_line_style'] == 2:
                        break
                i_content_start = i_content_start + 1 # 往上多找了一行
                
                for i_content_end in range(i,i+5,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        break
#                 i_content_end += 1 # 
                
                fc_dict[first_col_content] = []
#                 print(i_content_start,i_content_end+1)
                for i_content in range(i_content_start,i_content_end+1): # +1是因为range函数会去掉后边界
#                     print(excel.loc[i_content,1])
                    if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                        content = excel.loc[i_content,1]
                    else:
                        content = excel.loc[i_content,2]
                    if content == content: # 检测是否为nan
                        fc_dict[first_col_content].append(content)
                        
            if first_col_content == '原因':
                split_line_num = []
                for i_content_end in range(i,i_end,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        split_line_num.append(i_content_end + 1)
#                 split_line_num.append(i_content_end)
#                 print(split_line_num)
                fc_dict['起因'] = []
                fc_dict['诊断操作'] = []
                fc_dict['标准值'] = []
                fc_dict['步骤数'] = []
                for i_split in range(len(split_line_num)-1):
                    key = ""
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            key = excel.loc[i_content,1]
                            break
#                     value = ""
                    diagnosisActionList = []
                    standardValList = []
                    stepNumList = []
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        value_count_in_this_row = 0
                        for j in range(2,excel.shape[1]):
                            value_now = excel.loc[i_content,j]
                            if value_now != '.' and value_now != '·' and value_now == value_now:
#                                 value += str(value_now)
                                value_count_in_this_row += 1
                        if value_count_in_this_row == 1:
                            step = excel.loc[i_content,2]
                            stepNum = 0
                            match_result = re.match(r'(\d)\.(.*)', str(step))
                            if match_result is not None:
                                step = match_result.group(2).strip()
                                stepNum = match_result.group(1)
                            diagnosisActionList.append(step)
                            stepNumList.append(int(stepNum))
                        else:
                            standardVal_line = ""
                            for j in range(2,excel.shape[1]):
                                value_now = excel.loc[i_content,j]
                                if value_now != '.' and value_now != '·' and value_now == value_now:
                                    standardVal_line += str(value_now)
                            standardValList.append(standardVal_line)
                            
                    fc_dict['起因'].append(key)
                    fc_dict['诊断操作'].append(diagnosisActionList)
                    fc_dict['标准值'].append(standardValList)
                    fc_dict['步骤数'].append(stepNumList)
#             print(first_col_content)
            if first_col_content == '可能的原因和 标准':
#                 print(first_col_content)
                split_line_num = []
                for i_content_end in range(i,i_end,1):
                    cell = sheet.cell(i_content_end,1)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        split_line_num.append(i_content_end + 1)
#                 split_line_num.append(i_content_end)
#                 print(split_line_num)
                fc_dict['起因'] = []
                fc_dict['诊断操作'] = []
                fc_dict['标准值'] = []
                fc_dict['步骤数'] = []
                for i_split in range(len(split_line_num)-1):
                    key = ""
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        value_now = excel.loc[i_content,2]
                        if value_now != '.' and value_now != '·' and value_now == value_now:
                            key = value_now
                            break
#                         if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
#                             key = excel.loc[i_content,1]
#                             break
#                     value = ""
                    diagnosisActionList = []
                    standardValList = []
                    stepNumList = []
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        value_count_in_this_row = 0
                        for j in range(3,excel.shape[1]):
                            value_now = excel.loc[i_content,j]
                            if value_now != '.' and value_now != '·' and value_now == value_now:
#                                 value += str(value_now)
                                value_count_in_this_row += 1
                        if value_count_in_this_row == 1:
                            step = excel.loc[i_content,3]
                            stepNum = 0
                            match_result = re.match(r'(\d)\.(.*)', str(step))
                            if match_result is not None:
                                step = match_result.group(2).strip()
                                stepNum = match_result.group(1)
                            diagnosisActionList.append(step)
                            stepNumList.append(int(stepNum))
                        else:
                            standardVal_line = ""
                            for j in range(3,excel.shape[1]):
                                value_now = excel.loc[i_content,j]
                                if value_now != '.' and value_now != '·' and value_now == value_now:
                                    standardVal_line += str(value_now)
                            standardValList.append(standardVal_line)
                            
                    fc_dict['起因'].append(key)
                    fc_dict['诊断操作'].append(diagnosisActionList)
                    fc_dict['标准值'].append(standardValList)
                    fc_dict['步骤数'].append(stepNumList)
#     print(fc_dict)
    fc_dicts[fc_dict['故障代码']] = fc_dict
fc_dicts
    
    

i range in (0,49)
i range in (49,85)
i range in (85,108)
i range in (108,121)
i range in (121,145)
i range in (145,174)
i range in (174,203)
i range in (203,224)
i range in (224,241)
i range in (241,253)
i range in (253,269)
i range in (269,284)
i range in (284,302)
i range in (302,312)
i range in (312,327)
i range in (327,353)
i range in (353,363)
i range in (363,373)
i range in (373,412)
i range in (412,447)
i range in (447,481)
i range in (481,512)
i range in (512,531)
i range in (531,576)
i range in (576,596)
i range in (596,616)
i range in (616,631)
i range in (631,652)
i range in (652,669)
i range in (669,683)
i range in (683,717)
i range in (717,737)
i range in (737,750)
i range in (750,1079)
i range in (1079,1337)
i range in (1337,1407)
i range in (1407,1426)
i range in (1426,1493)
i range in (1493,1529)
i range in (1529,1590)
i range in (1590,1625)
i range in (1625,1694)
i range in (1694,1740)
i range in (1740,1754)
i range in (1754,1765)
i range in (1765,1793)
i range in (179

{'H-1': {'故障代码': 'H-1',
  '故障': '整个工作装置、回转、和行走的速度或动力低',
  '相关信息': ['·   在工作模式为快速模式（P）时，进行所有故障诊断。'],
  '起因': ['伺服活塞故障', '柱塞泵故障'],
  '诊断操作': [['伺服活塞可能有故障。直接检查它。'], ['原因可能是柱塞泵性能下降、失灵、或内部故障。']],
  '标准值': [[], []],
  '步骤数': [[0], [0]]},
 'H-2': {'故障代码': 'H-2',
  '故障': '发动机转速极速下降或发动机突然熄火',
  '相关信息': ['·   在工作模式为快速模式（P）时，进行所有故障诊断。'],
  '起因': ['LS 阀调整故障或失灵', '伺服装置节流孔或滤清器堵 塞', '伺服活塞故障'],
  '诊断操作': [['★ 关闭发动机，做好准备，然后在发动机全速时进行故障诊断。',
    nan,
    nan,
    nan,
    '调整后，如果油压不能变为正常，LS 阀则可能失灵或有内部故障。 直接检查 LS 阀。'],
   ['泵内伺服装置的节流孔或滤清器可能堵塞，直接检查它们。'],
   ['伺服活塞可能有故障。直接检查它。']],
  '标准值': [['测定油压所有操纵杆处于 中位无载荷行走', '泵排放压力1', 'PC 阀输出压力压力几乎相同约 0.6'], [], []],
  '步骤数': [[0, 0, 0, 0, 0], [0], [0]]},
 'H-3': {'故障代码': 'H-3',
  '故障': '工作装置、回转和行走系统不工作',
  '相关信息': ['·   工作模式为快速模式（P）时，进行所有故障诊断。'],
  '起因': ['柱塞泵故障', '减震器故障'],
  '诊断操作': [['柱塞泵可能故障或有内部故障。按照下列方法进行检查。',
    '1）拆下泵输出压力检测螺塞（前后）。',
    '2）以飞轮（手动）运转发动机。',
    '3）如果油流出泵输出压力检测口，则情况正常。'],
   ['由于减震器内部故障，泵轴可能不转动。 检查减震器。']],
  '标准值': [[], []],
  '步骤数': [[0, 0, 0, 0

## 将故障值从字典形式转化为关系表形式

In [37]:
merge = pd.DataFrame(columns=name_to_en.keys())
for fc_dict_oral in fc_dicts.values():
#     print(fc_dict)
    fc_dict = copy.deepcopy(fc_dict_oral)
    list_keys = []
    for key in fc_dict.keys():
#         print(type(fc_dict[key]))
        if isinstance(fc_dict[key], list):
            list_keys.append(key)
#     print(list_keys)
    for key in list_keys:
        fc_dict.pop(key, None)
    single_fc_df = pd.DataFrame(fc_dict, index=[0])
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    # fc_dict_list = {}
    for key in list_keys:
        if key == '诊断操作' or key == '标准值' or key == '步骤数':
            continue
        fc_dict_list = {}
#         print(fc_dict)
        fc_dict_list['故障代码'] = [fc_dict['故障代码']] * len(fc_dict[key])
        fc_dict_list[key] = fc_dict[key]
#         print(fc_dict_list)
#         print(pd.DataFrame(fc_dict_list))
        tmp_df = pd.DataFrame(fc_dict_list)
    #     a = pd.concat((a,b))
        single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    cause_dict = {}
    cause_dict['起因'] = []
    cause_dict['诊断操作'] = []
    cause_dict['步骤数'] = []
#     print(fc_dict)
    if '起因' not in fc_dict.keys():
        print(fc_dict)
    else:
        for i in range(len(fc_dict['起因'])):
            cause = fc_dict['起因'][i]
            diagnosisAction = fc_dict['诊断操作'][i]
            stepNum = fc_dict['步骤数'][i]
    #         print("debug:cause=",cause)
    #         print("debug:diagnosisAction=",diagnosisAction)
            cause_dict['起因'] += [cause] * len(diagnosisAction)
            cause_dict['诊断操作'] += diagnosisAction
            cause_dict['步骤数'] += stepNum
    #         cause_dict['起因'].append(cause)
    #         cause_dict['诊断操作'].append(diagnosisAction)
    #     print(cause_dict)
        if len(cause_dict) == 1:
            tmp_df = pd.DataFrame(cause_dict, index=[0])
        else:
            tmp_df = pd.DataFrame(cause_dict)
        single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    merge = pd.concat((merge,single_fc_df), sort=True)
#     print("*"*80)
merge

{'故障代码': 'E-36', '故障': 'KOMTRAX 系统不正常工作', '相关信息': ['· 如果 KOMTRAX 系统管理员要求检查机器系统中是否有故障，则进行下列故障诊断。', '· 即使 KOMATRAX 系统有故障时，机器似乎没有问题。', '· 检查机器监控器服务菜单显示屏上的故障诊断结果。']}


,报警级别,控制器的反应,故障,故障代码,故障内容,故障名称,故障部位系统,机器发生的故障,标准值,步骤数,相关信息,诊断操作,起因,附图
0,NaN,NaN,整个工作装置、回转、和行走的速度或动力低,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,· 在工作模式为快速模式（P）时，进行所有故障诊断。,NaN,NaN,NaN
0,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,伺服活塞故障,NaN
1,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,柱塞泵故障,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,伺服活塞可能有故障。直接检查它。,伺服活塞故障,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,原因可能是柱塞泵性能下降、失灵、或内部故障。,柱塞泵故障,NaN
0,NaN,NaN,发动机转速极速下降或发动机突然熄火,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,· 在工作模式为快速模式（P）时，进行所有故障诊断。,NaN,NaN,NaN
0,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LS 阀调整故障或失灵,NaN
1,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,伺服装置节流孔或滤清器堵 塞,NaN


In [39]:
merge = merge.rename(columns=name_to_en)
merge.to_csv("./data/decoration/manual_fc.csv",quoting=1,sep=';',index=False,encoding='utf-16')
merge

,alertlevel,controllerreaction,faultname,faultcode,faultcontent,faultname,monitorsystem,machinefault,standardval,stepnum,relatedinfo,diagnosisaction,cause,image
0,NaN,NaN,整个工作装置、回转、和行走的速度或动力低,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,· 在工作模式为快速模式（P）时，进行所有故障诊断。,NaN,NaN,NaN
0,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,伺服活塞故障,NaN
1,NaN,NaN,NaN,H-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,柱塞泵故障,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,伺服活塞可能有故障。直接检查它。,伺服活塞故障,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,原因可能是柱塞泵性能下降、失灵、或内部故障。,柱塞泵故障,NaN
0,NaN,NaN,发动机转速极速下降或发动机突然熄火,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,· 在工作模式为快速模式（P）时，进行所有故障诊断。,NaN,NaN,NaN
0,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LS 阀调整故障或失灵,NaN
1,NaN,NaN,NaN,H-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,伺服装置节流孔或滤清器堵 塞,NaN
